In [ ]:
# first we load the data; note this is a subset of the mock used for the data
# credits to Francesco Sorrenti (https://fsorrenti.github.io/) for the mock generation

In [19]:
import numpy
from velocemu.emulator import IntegralEmu

/home/users/p/piras/veloce/velocemu/emulator.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [18]:
z_values = numpy.load('../../curl_project/z_mock_1457.npy').astype(float)[:500]
mu_values = numpy.load('../../curl_project/mu_mock_1457.npy').astype(float)[:500]
error_covariance = np.load("../../curl_project/cov_carr_removed_ceph_1457.npy")[:500, :500]
z_z_alpha=numpy.load('/home/users/p/piras/curl_project/z_z_alphas/z_mock_z_mock_alpha_no_cepheid_cleaned.npy')[:125250]

In [21]:
integrator = IntegralEmu(z_z_alpha)

unique_rows = numpy.load('./test_unique_rows.npy')
inverse_indices = numpy.load('./test_inverse_indices.npy')
condition = numpy.load('./test_condition.npy')

print(np.sum(condition), np.sum(~condition))

2025-07-08 23:51:40.327885: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


500 124750


In [ ]:
def log_prior(params):
    """
    Compute the log-prior for 3 parameters with uniform priors.
    """
    # Define the parameter ranges for uniform priors
    ranges = np.array([
        (0.1, 0.5),  # Range for Omat
        (50., 100.),    # Range for H0
        (1.5, 4.5),    # Range for lnAs
    ])

    # Compute log-prior for each parameter
    lower_bounds, upper_bounds = ranges[:, 0], ranges[:, 1]
    inside_bounds = (params >= lower_bounds) & (params <= upper_bounds)

    # Use np.where to avoid TracerBoolConversionError
    log_prior_value = np.where(
        np.all(inside_bounds),
        np.sum(-np.log(upper_bounds - lower_bounds)),
        -np.inf
    )

    return log_prior_value


def log_prob_(parameters):
    """The total log-probability, namely the sum of log-prior and log-likelihood"""

    log_prior_value = log_prior(parameters)    
    
    def compute_likelihood(_):
        Omat, H0, lnAs = parameters 
        s_estimated, prefactors1 = s_estimator(z_current, mudata, cephdata, is_caldata, H01, Omat, 0.)
        
        _, prefactors2 = s_estimator(emuz, mudata, cephdata, is_caldata, H02, Omat, 0.)
        # Use the outer product to get the full prefactor matrix
        prefactor_matrix = np.outer(prefactors2, prefactors2)

        cova = reconstruct_symmetric_matrix_from_lower_diagonal(integrator.predict(np.array([Omat,H0,lnAs]), 
                                                                                   unique_rows, 
                                                                                   inverse_indices, condition), 
                                                                500)
        f0 = np.real(fz_jax(0, Omat))
        cova *= f0**2
        
        full_covariance=error_covariance+cova/prefactor_matrix

        if is_pd(full_covariance)==False:
            return -np.inf

        log_likelihood = multivariate_normal.logpdf(np.zeros(500), mean=s_estimated/prefactors1, 
                                                    cov=np.array(full_covariance))
    
        return log_prior_value + log_likelihood

    # to make everything compatible with JAX
    return jax.lax.cond(
        log_prior_value > -np.inf,  # Check if prior is valid
        compute_likelihood,      # Compute likelihood if prior is valid
        lambda _: -np.inf,      # Return -inf if prior is invalid
        operand=None,
    )

In [ ]:
zmock = numpy.load('./z_mock_1457.npy').astype(float)
mudata = numpy.load('./mu_mock_1457.npy').astype(float)   # Convert column 1 to float
cephdata = np.zeros_like(mudata) # there are no cepheids in this example
is_caldata = np.zeros_like(mudata)  # there are no cepheids in this example

if analysis == 'zmock':
    z_current = zmock

if emuanalysis == 'zmock':    
    emuz = zmock
    # z_mock_z_mock_alpha_cleaned_1106_seed_40_20_z_hel_lower_plus_diagonal_67_035_removing_cepheid_corrected.npy
    input_alpha_z_z=numpy.load('/home/users/p/piras/curl_project/z_z_alphas/z_mock_z_mock_alpha_no_cepheid_cleaned.npy')


#this is already all the matrix
# covariance_pantheon_cleaned_statistical_systematic_filtered_5e5_1106_no_cepheid_correct.npy
covf = np.load("cov_carr_removed_ceph_1457.npy")
error_covariance = covf

In [ ]:
def log_prior(params):
    """
    Compute the log-prior for 6 parameters with uniform priors.

    Parameters:
        params (array-like): An array of 6 parameters [p1, p2, p3, p4, p5, p6].

    Returns:
        float: The value of the log-prior.
    """
    # Define the parameter ranges for uniform priors
    ranges = np.array([
        (0.1, 0.5),  # Range for p1
        #(0.1, 0.5),  # Range for p2
        (60., 85.),    # Range for p3
        #(60., 85.),    # Range for p3
        #(1.9, 3.9),  # Range for p4
        #(-6, 12.),
        (2., 6.),
        (-0.5, 0.5)  # Range for p6
    ])

    # Compute log-prior for each parameter
    lower_bounds, upper_bounds = ranges[:, 0], ranges[:, 1]
    inside_bounds = (params >= lower_bounds) & (params <= upper_bounds)

    # Use np.where to avoid TracerBoolConversionError
    log_prior_value = np.where(
        np.all(inside_bounds),
        np.sum(-np.log(upper_bounds - lower_bounds)),
        -np.inf
    )

    return log_prior_value


def log_prob_(parameters):

    log_prior_value = log_prior(parameters)    
    
    def compute_likelihood(_):
        Omat, H0, log10eps, dM = parameters
        #log10eps = 4.#5.2
        eps = 10**log10eps
        s_estimated, prefactors1 = s_estimator(z_current, mudata, cephdata, is_caldata, H0, Omat, dM)
        
        _, prefactors2 = s_estimator(emuz, mudata, cephdata, is_caldata, H0, Omat, dM)
        # Use the outer product to get the full prefactor matrix
        prefactor_matrix = np.outer(prefactors2, prefactors2)

        cova = reconstruct_symmetric_matrix_from_lower_diagonal(integrator.predict(np.array([Omat,H0,3.047]), # 3.047
                                                                                   unique_rows, 
                                                                                   inverse_indices, condition), 
                                                                1624)
        f0 = np.real(fz_jax(0, Omat))
        cova *= f0**2
        
        #ceph_mask = np.outer(1-is_caldata, 1-is_caldata) # OR mask
        full_covariance=error_covariance+eps*np.eye(1624)/prefactor_matrix+cova/prefactor_matrix

        if is_pd(full_covariance)==False:
            return -np.inf

        log_likelihood = multivariate_normal.logpdf(np.zeros(1624), mean=s_estimated/prefactors1, 
                                                    cov=np.array(full_covariance))
    
        return log_prior_value + log_likelihood

    return jax.lax.cond(
        log_prior_value > -np.inf,  # Check if prior is valid
        compute_likelihood,      # Compute likelihood if prior is valid
        lambda _: -np.inf,      # Return -inf if prior is invalid
        operand=None,
    )